In [4]:
pip install opencv-python
pip install spacy
pip install pytesseract
pip install keras
pip install tensorflow
pip install opencv-python-headless numpy
pip install torch torchvision
!python -m spacy download en_core_web_sm

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/66/82/564168a349148298aca281e342551404ef5521f33fba17b388ead0a84dc5/opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 38.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract' 

In [3]:
!export PATH=$PATH:/opt/homebrew/bin/tesseract

In [62]:
import pandas as pd
import numpy as np
import pytesseract
import spacy
import cv2
import os

from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Model
from keras.layers import Input, Dense, Concatenate

## Exploring Datasets

In [38]:
sample_df = pd.read_csv('Sample.csv')
ground_truth_df = pd.read_csv('ground-truth.csv')

In [54]:
sample_df

,creative_data_id,creative_data_title,creative_data_description,creative_data_duration,creative_data_lifetime_spend_estimated,creative_data_lifetime_airings_count,creative_data_airing_date_first_et,creative_data_airing_date_last_et,speech
0,2194673,30s Kim's Discount - 2194673,Kim is going for the State Farm Drive Safe & S...,30,29789808.73,13949,2019-04-06T22:19:06-04:00,2020-08-04T18:42:50-04:00,"So Kim, you going for a big drive safe and sav..."
1,2142915,30s New Flat - 2142915,Uncomfortable with her shabby apartment and ro...,30,5423001.70,10132,2019-03-04T06:49:02-05:00,2021-08-03T11:12:36-04:00,Check your credit scores for free and learn ho...
2,1702851,30s Most Pills Don't Finish the Job - 1702851,Flonase guesses you wouldn't accept an incompl...,30,23072716.78,8491,2018-02-19T07:42:14-05:00,2020-06-01T04:22:16-04:00,You wouldn't accept an incomplete job from any...
3,1671980,30s Box Vlog - 1671980,The Progressive Box starts his own vlog as he ...,30,44909836.61,7766,2018-01-01T00:17:22-05:00,2018-06-10T15:53:15-04:00,Subword. It's the box with 30% savings for saf...
4,1749291,30s Scars: One Way Out [T1] - 1749291,Chevrolet owners tell stories of how their Sil...,30,3490623.10,5260,2018-05-01T07:07:56-04:00,2019-06-30T23:24:28-04:00,I got scar tissue there. Same thing with any D...
...,...,...,...,...,...,...,...,...,...
145,2755227,30s We Are Here For You [T2] - 2755227,Toyota says its family stretches to every corn...,30,1581.88,9,2020-04-01T11:59:46-04:00,2020-04-07T19:29:18-04:00,Toyota's family stretches to every corner of t...
146,2259242,30s Born to Drive Sales Event: Born to Drive [...,MINI USA says its vehicles weren't born ordina...,30,43704.13,8,2019-06-01T12:09:22-04:00,2019-06-02T01:42:24-04:00,Minnie wasn't born ordinary. Minnie was born e...
147,3124938,30s Big Finish 2020: One of Our Own [T2] - 312...,Jeep believes that owners of its vehicles are ...,30,1501.33,8,2020-12-19T19:28:51-05:00,2020-12-28T04:00:20-05:00,Jeep is more than a legendary Suv brand. It's ...
148,3264190,30s Truck Month: Final Days: One Show You Don'...,With Ford Truck Month coming to its final days...,30,3802.55,8,2021-03-21T01:09:37-04:00,2021-03-21T23:56:57-04:00,"Texas, the final days are here and that means ..."


In [53]:
ground_truth_df

,Timestamp,creative_data_id,"Is there a call to go online (e.g., shop online, visit the Web)?","Is there online contact information provided (e.g., URL, website)?","Is there a visual or verbal call to purchase (e.g., buy now, order now)?","Does the ad portray a sense of urgency to act (e.g., buy before sales ends, order before ends)?","Is there an incentive to buy (e.g., a discount, a coupon, a sale or ""limited time offer"")?","Is there offline contact information provided (e.g., phone, mail, store location)?",Is there mention of something free?,"Does the ad mention at least one specific product or service (e.g., model, type, item)?",...,Was there a famous person in this ad?,"If yes to the above, write the name of the famous person, if known.",What happened in this ad? (Answer in 2-3 sentences each),What was/were the company's goal(s) with this ad? Choose (potentially multiple) from:,How successful was the ad in achieving its goal(s)?,"How much did you like the ad? (1. Strongly dislike, 2. Dislike, 3. Neither Like or Dislike, 4. Like, 5. Strongly Like)","What was the slogan presented in the ad, if any?","After addressing the specific survey items, write a general description of the ad. You can use answers to the questions above to formulate your answer. Your description should include:\nBrand and Product Identification: \nSpecify the brand and whether a product is being advertised. (1 sentence)\nVisual Elements: Describe what is seen on the screen, including setting, characters, and any text or graphics. (max 2 sentences)\nAuditory Elements: Note what is heard, such as dialogue, voice-over, music, or sound effects. (max 2 sentences)\n",Any additional feedback or things we should be aware of?,Please enter the video identifier one more time (e.g. 123456789.mp4)
0,5/16/2024 8:00:14,1471363,No,Yes,No,No,No,No,No,Yes,...,No,NaN,During this ad we had a man and a woman taking...,Change how consumers feel about the product/br...,3,3,NaN,Product Identification: Mini Countryman SUV\nM...,I believe the text legibility should be improv...,1471363.mp4
1,5/23/2024 2:35:55,1471363,No,Yes,No,No,No,No,No,Yes,...,No,NaN,We watch Mini USA new Countryman driver around...,Directly persuade consumers to purchase,2,2,n/a,Mini USA is advertising their new Countryman c...,NaN,1471363.mp4
2,5/15/2024 18:09:54,1488315,No,No,No,No,No,No,No,No,...,No,NaN,The end of the world seems to have been brough...,"Increase awareness of product/brand, Make cons...",3,3,"Their usual slogan was not presented, but ""Tom...",The brand being advertised is Jose Cuervo tequ...,NaN,1488315.mp4
3,5/16/2024 6:18:34,1488315,No,No,No,No,No,No,No,Yes,...,No,NaN,This ad is set in a bar in the middle of a des...,"Increase awareness of product/brand, Change ho...",4,4,"Cuervo, ""Tomorrow is Overrated""","This ad is for the brand Jose Cuervo, advertis...",NaN,1488315.mp4
4,5/24/2024 10:00:26,1488315,No,No,No,No,No,No,No,Yes,...,No,NaN,It started with a disaster occurring and the n...,Increase awareness of product/brand,4,3,Tomorrow is overrated,The brand being advertised was Jose Cuervo teq...,NaN,1488315.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,5/24/2024 17:09:00,3415261,No,Yes,No,No,No,No,No,Yes,...,No,na,"In the ad, a bmw car technitian is shown fixin...","Increase awareness of product/brand, Increase ...",5,5,na,"The brand is bmw, no specific model was shown....",na,3415261.mp4
445,5/28/2024 4:58:33,3415261,No,Yes,Yes,No,No,No,No,Yes,...,No,NaN,The ad is describing their BMW Certified model...,Change how consumers feel about the product/br...,4,3,NaN,The brand is BMW and there is a product being ...,NaN,3415261.mp4
446,5/25/2024 8:53:24,3422482,No,No,Yes,Yes,Yes,No,No,No,...,No,none,The ad starts off saying for those who care ab...,"Directly persuade consumers to purchase, Incre...",5,5,no,\n1. They are promoting their Golden Opportuni...,no,3422482
447,5/27/2024 0:08:51,3422482,No,No,Yes,Yes,No,No,No,Yes,...,No,NaN,A woman

In [41]:
# Checking for missing values
sample_df.isnull().sum()

creative_data_id                          0
creative_data_title                       0
creative_data_description                 0
creative_data_duration                    0
creative_data_lifetime_spend_estimated    0
creative_data_lifetime_airings_count      0
creative_data_airing_date_first_et        0
creative_data_airing_date_last_et         0
speech                                    0
dtype: int64

In [42]:
ground_truth_df.isnull().sum()

Timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         0
creative_data_id                                                                                                                                                                                                                                                                                                                                                                                                                                                    

### Extracting Textual Features

In [45]:
ground_truth_df.columns

Index(['Timestamp', 'creative_data_id',
       'Is there a call to go online (e.g., shop online, visit the Web)? ',
       'Is there online contact information provided (e.g., URL, website)? ',
       'Is there a visual or verbal call to purchase (e.g., buy now, order now)?',
       'Does the ad portray a sense of urgency to act (e.g., buy before sales ends, order before ends)? ',
       'Is there an incentive to buy (e.g., a discount, a coupon, a sale or "limited time offer")? ',
       'Is there offline contact information provided (e.g., phone, mail, store location)?',
       'Is there mention of something free? ',
       'Does the ad mention at least one specific product or service (e.g., model, type, item)? ',
       'Is there any verbal or visual mention of the price?',
       'Does the ad show the brand (logo, brand name) or trademark (something that most people know is the brand) multiple times?\n\nFor example, Nike ads often have the "swoosh" logo prominently displayed on shoe

In [43]:
sample_df.columns

Index(['creative_data_id', 'creative_data_title', 'creative_data_description',
       'creative_data_duration', 'creative_data_lifetime_spend_estimated',
       'creative_data_lifetime_airings_count',
       'creative_data_airing_date_first_et',
       'creative_data_airing_date_last_et', 'speech'],
      dtype='object')

In [8]:
#Extracting Features from Descriptions and speech
nlp = spacy.load("en_core_web_sm")

# Example: Process a single description
description = sample_df['creative_data_description'].iloc[0]
doc = nlp(description)

# Extract entities and keywords
entities = [(ent.text, ent.label_) for ent in doc.ents]
keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

print("Entities:", entities)
print("Keywords:", keywords)

Entities: [('Kim', 'PERSON'), ('the State Farm Drive Safe & Save', 'ORG'), ('State Farm', 'ORG'), ('up to 30 percent', 'PERCENT')]
Keywords: ['Kim', 'go', 'State', 'Farm', 'Drive', 'Safe', 'Save', 'Discount', 'app', 'say', 'vigilant', 'refuse', 'speed', 'meeting', 'start', 'need', 'use', 'restroom', 'go', 'labor', 'refuse', 'let', 'anybody', 'mess', 'discount', 'State', 'Farm', 'say', 'discount', 'percent']


In [85]:
#Extracting Features from On-Screen Text - We will use Optical Character Recognition (OCR) to extract text from video frames.

# Function to extract text from a video frame
def extract_text_from_frame(frame):
    pil_image = Image.fromarray(frame)
    text = pytesseract.image_to_string(pil_image)
    return text

# Function to extract key frames from a video
def extract_key_frames(video_path, num_frames=5):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)
    key_frames = []

    for frame_idx in frame_indices:
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = video.read()
        if ret:
            key_frames.append(frame)

    video.release()
    return key_frames

In [86]:
# Extract key frames and text from the example video
video_path = 'sample/1471363.mp4'
key_frames = extract_key_frames(video_path)
frame_texts = [extract_text_from_frame(frame) for frame in key_frames]

print("Extracted Text from Frames:")
for i, text in enumerate(frame_texts):
    print(f"Frame {i+1}: {text}")

Extracted Text from Frames:
Frame 1: a 2 | Fr

Frame 2: ADD

Frame 3: 
Frame 4: 
Frame 5: miniusa.com/newcountryman

© 2017 MINI USA, « division of BMW of North America, LLC. The MINI name, model names, and logo are registered trademarks.




### Exploring the Video Files

In [25]:
# Function to extract video information
def get_video_info(video_path):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        return None
    info = {
        'Frame Width': int(video.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'Frame Height': int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        'Frame Rate': video.get(cv2.CAP_PROP_FPS),
        'Frame Count': int(video.get(cv2.CAP_PROP_FRAME_COUNT)),
        'Duration (seconds)': int(video.get(cv2.CAP_PROP_FRAME_COUNT)) / video.get(cv2.CAP_PROP_FPS)
    }
    video.release()
    return info

In [26]:
# Example
video_path = 'sample/1471363.mp4'
video_info = get_video_info(video_path)
print(f"Information for video {os.path.basename(video_path)}:")
print(video_info)

Information for video 1471363.mp4:
{'Frame Width': 1280, 'Frame Height': 720, 'Frame Rate': 29.97002997002997, 'Frame Count': 900, 'Duration (seconds)': 30.03}


## Preprocessing text data

In [49]:
# Merge datasets on 'creative_data_id'
data = pd.merge(sample_df, ground_truth_df, on='creative_data_id')

In [50]:
# Combine text columns
data['combined_text'] = data['creative_data_description'].fillna('') + " " + data['speech'].fillna('')

In [57]:
# Text vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_text = vectorizer.fit_transform(data['combined_text'])

In [73]:
def list_video_files(directory):
    supported_formats = ('.mp4', '.avi', '.mov')  # Add or remove video formats as needed
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(supported_formats)]

video_files = list_video_files('sample/')

In [76]:
import cv2
import numpy as np

def get_average_frame_color(video_path):
    cap = cv2.VideoCapture(video_path)
    colors = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Calculate the average color of each frame
        average_color_per_row = np.average(frame, axis=0)
        average_color = np.average(average_color_per_row, axis=0)
        colors.append(average_color)

    cap.release()
    # Compute the overall average color
    if colors:
        average_color = np.mean(colors, axis=0)
    else:
        average_color = [0, 0, 0]  # Default if no frames are found
    
    return average_color.tolist()

In [77]:
def detect_logo(video_path):
    # Mock function for logo detection, always returns False
    return False

In [78]:
def extract_video_features(video_path):
    average_color = get_average_frame_color(video_path)
    logo_detected = detect_logo(video_path)  # This would normally use a real detection model
    return {'average_color': average_color, 'logo_detected': logo_detected}

In [79]:
video_features = {video: extract_video_features(video) for video in video_files}

In [80]:
import pandas as pd

# Create a DataFrame from the dictionary
df_video_features = pd.DataFrame.from_dict(video_features, orient='index').reset_index()
df_video_features.columns = ['video_path', 'average_color', 'logo_detected']

# Save to CSV
df_video_features.to_csv('video_features.csv', index=False)

In [81]:
df_video_features

,video_path,average_color,logo_detected
0,sample/2592911.mp4,"[56.733730185740725, 59.22614815882621, 58.805...",False
1,sample/3209706.mp4,"[87.51566840880594, 85.90027477695791, 99.8321...",False
2,sample/2807978.mp4,"[102.29124417635136, 76.21325356684893, 115.86...",False
3,sample/2962688.mp4,"[136.92518890802864, 146.8683808001262, 142.51...",False
4,sample/2530396.mp4,"[53.007094843981186, 52.03855855569434, 48.341...",False
...,...,...,...
145,sample/2314704.mp4,"[43.49285312107771, 54.34685001630825, 63.2750...",False
146,sample/1744482.mp4,"[50.237904887809385, 84.60645825608697, 102.58...",False
147,sample/2505718.mp4,"[56.70038286554789, 60.26348673683447, 58.8523...",False
148,sample/1710993.mp4,"[138.95389957422785, 96.19722555709075, 54.856...",False


In [88]:
# Example usage on a DataFrame
data['video_features'] = data['sample'].apply(extract_video_features)

KeyError: 'sample'

## Model Development

In [89]:
# Assuming X_text is sparse, convert it to dense
X_text_dense = X_text.todense()

# Convert video features to a suitable format
video_features = np.array(data['video_features'].tolist())

# Inputs
text_input = Input(shape=(X_text_dense.shape[1],))
video_input = Input(shape=(video_features.shape[1],))

# Layers
text_layer = Dense(128, activation='relu')(text_input)
video_layer = Dense(32, activation='relu')(video_input)
concat_layer = Concatenate()([text_layer, video_layer])
output_layer = Dense(21, activation='sigmoid')(concat_layer)  # 21 questions, binary answers

# Model
model = Model(inputs=[text_input, video_input], outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training
model.fit([X_text_dense, video_features], data[questions], epochs=10, batch_size=16)

KeyError: 'video_features'

## Evaluation

In [ ]:
from sklearn.metrics import classification_report

# Mock predictions for demonstration
predictions = model.predict([X_text_dense, video_features])
predictions = (predictions > 0.5).astype(int)  # Thresholding to get binary values

# Calculate metrics
print(classification_report(data[questions], predictions))  # Assuming 'questions' is a list of your 21 target columns